In [1]:
import tensorflow as tf

In [2]:
import os
os.chdir('../')

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [4]:
from src.classifier.constants import *
from src.classifier.utils import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = Config_File_Path,
                 params_filepath= Params_File_Path
                 ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Kidney-CT-Scan-data")

        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir= Path(training.root_dir),
            trained_model_path= Path(training.trained_model_path),
            updated_base_model_path= Path(prepare_base_model.updated_base_model_path),
            training_data= Path(training_data),
            params_epochs= params.EPOCHS,
            params_batch_size= params.BATCH_SIZE,
            params_is_augmentation= params.AUGMENTATION,
            params_image_size= params.IMAGE_SIZE
        )

        return training_config

In [6]:
class Trainig:
    def __init__(self, config: TrainingConfig):
        self.config= config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):
        datagenerator_kwargs = dict (
            rescale = 1./255,
            validation_split = 0.20
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = 'bilinear'
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory =self.config.training_data,
            subset= 'validation',
            shuffle= False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range= 40,
                horizontal_flip= True,
                width_shift_range= 0.2,
                height_shift_range= 0.2,
                shear_range= 0.2,
                zoom_range= 0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator
        
        self.train_generator = train_datagenerator.flow_from_directory(
            directory= self.config.training_data,
            subset= 'training',
            shuffle= 'True',
            **dataflow_kwargs
        )
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
    

    def train(self):
        self.steps_pre_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs= self.config.params_epochs,
            steps_per_epoch= self.steps_pre_epoch,
            validation_steps= self.validation_steps,
            validation_data= self.valid_generator,
        )

        self.save_model(
            path= self.config.trained_model_path,
            model= self.model
        )

In [7]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Trainig(config= training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
except Exception as e:
    raise e

Found 535 images belonging to 2 classes.
Found 2147 images belonging to 2 classes.
Epoch 1/5
134/134 [==============================] - 1062s 8s/step - loss: 11.0943 - accuracy: 0.5847 - val_loss: 3.2544 - val_accuracy: 0.6534
Epoch 2/5
134/134 [==============================] - 1042s 8s/step - loss: 7.4853 - accuracy: 0.6603 - val_loss: 1.3653 - val_accuracy: 0.6780
Epoch 3/5
134/134 [==============================] - 1048s 8s/step - loss: 5.1950 - accuracy: 0.7180 - val_loss: 3.3376 - val_accuracy: 0.7595
Epoch 4/5
134/134 [==============================] - 1046s 8s/step - loss: 3.7528 - accuracy: 0.7508 - val_loss: 13.4175 - val_accuracy: 0.6004
Epoch 5/5
134/134 [==============================] - 1040s 8s/step - loss: 3.6198 - accuracy: 0.7799 - val_loss: 8.4921 - val_accuracy: 0.4640
